# RaceVision : F1 Lap-by-Lap Race Prediction
Data are from Ergast DB : http://ergast.com/mrd/db/  

This model uses an LSTM to predict the positions, laptimes and pitstops for 20 drivers. 
Features to add : 
- Weather
- Tyres and track types
- (if rules chage, fuel weight impact on lap time and pit stop)

## Initialization - OK

In [1]:
#%pip install ipywidgets
#%pip install matplotlib
import matplotlib.pyplot as plt
#%pip install pandas
import pandas as pd
#%pip install numpy
import numpy as np
#%pip install tqdm
from tqdm.auto import tqdm
#%pip install os
import os
#%pip uninstall -y torch torchvision torchaudio
#%pip install --no-cache-dir --force-reinstall \
#    torch torchvision torchaudio \
#    --index-url https://download.pytorch.org/whl/cu128
import torch, torch.nn as nn, torch.optim as optim, torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
print("cuda available?", torch.cuda.is_available())
print("cuda count   ", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU name     ", torch.cuda.get_device_name(0))
#%pip install math
import math
#%pip install pathlib
from pathlib import Path

cuda available? True
cuda count    1
GPU name      NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
versionId = "1.0"

In [3]:
# Directory where the raw CSV and processed data live
db_dir = Path(r"E:\Dropbox\Informatique\Holberton\F1_Project\db")

In [4]:
# The time format in the Ergast database is MM:SS.ms
# For a better work, we split aroud the ":" and express it in seconds (float)
def time_to_int(t):
  # If input is already float type, return as-is
  if (t == float):
    return t
  t2 = str(t)
  ts = t2.rsplit(':')
  # Handle missing values
  if ('\\N' in t2):
    return None # missing data
  if (not '.' in t2):
    return None # unexpected format
  # Convert minutes and seconds to total seconds
  if (len(ts) > 1):
    return int(ts[0]) * 60 + float(ts[1])
  else:
    return float(ts[0])

### Functions used to get information from .csv files

In [5]:
# Load metadata tables once for lookup
races = pd.read_csv(db_dir / 'races.csv')
circuits = pd.read_csv(db_dir / 'circuits.csv')
drivers = pd.read_csv(db_dir / 'drivers.csv')
constructors = pd.read_csv(db_dir / 'constructors.csv')
status = pd.read_csv(db_dir / 'status.csv')

def race_info(raceId):
  # Retrieve year, round, and circuitId for a given raceId
  _races = races
  _r = _races.query(f'raceId  == {raceId}')
  if (_r.empty):
    return None, None, None
  _year = _r['year'].item()
  _round = _r['round'].item()
  _circuitId = _r['circuitId'].item()
  return _year, _round, _circuitId

def circuit_info(circuitId):
  # Retrieve circuit name, location, country
  _circuits = circuits
  _c = _circuits.query(f'circuitId  == {circuitId}')
  if (_c.empty):
    return None, None, None
  _name = _c['name'].item()
  _location = _c['location'].item()
  _country = _c['country'].item()
  return _name, _location, _country

def driver_info(id):
  # Retrieve driver metadata (number, code, name, dob, nationality)
  _drivers = drivers
  _d = _drivers.query(f'driverId  == {id}')
  if (_d.empty):
    return None, None, None, None, None, None
  _number = _d['number'].item()
  _code = _d['code'].item()
  _forename = _d['forename'].item()
  _surname = _d['surname'].item()
  _dob = _d['dob'].item()
  _nationality = _d['nationality'].item()
  return _number, _code, _forename, _surname, _dob, _nationality

def constructor_info(id):
  # Retrieve constructor name and nationality
  _constructor = constructors
  _c = _constructor.query(f'constructorId  == {id}')
  if (_c.empty):
    return None, None
  _name = _c['name'].item()
  _nationality = _c['nationality'].item()
  return _name, _nationality

def status_info(id):
  # Retrieve textual status for a given statusId
  _status = status
  _s = _status.query(f'statusId == {id}')
  if (_s.empty):
    return None
  _sstr = _s['status'].item()
  return _sstr

## Data preparation - OK

Each csv file contains informations for one race
Each row in each file contains informations for one lap. 

We are using the files in <db_dir>/races/<year> generated by 1-DataPreparation.ipynb .  

In [ ]:
# Creation of each year's folder
years = range(2018, 2024)   #range(2001, 2024)
for year in years:
  if not os.path.exists(db_dir / f'races_npy/{year}'):
        os.makedirs(db_dir / f'races_npy/{year}')

### Status Embedding

In [7]:
stat_emb = [
  [4.0, 3.0, 130.0], # Accident/Collision
  [22.0, 5.0, 10.0, 23.0, 44.0, 47.0, 30.0, 32.0, 8.0, 38.0, 43.0, 85.0, 9.0, 86.0, 6.0, 2.0, 7.0, 87.0, 71.0, 41.0, 46.0, 37.0, 65.0, 78.0, 25.0, 74.0, 75.0, 26.0, 51.0, 40.0, 79.0, 36.0, 83.0, 80.0, 21.0, 69.0, 72.0, 70.0, 27.0, 60.0, 63.0, 29.0, 64.0, 66.0, 56.0, 59.0, 61.0, 42.0, 39.0, 48.0, 49.0, 34.0, 35.0, 28.0, 24.0, 33.0, 129.0, 76.0, 91.0, 131.0, 101.0, 132.0, 135.0,  84.0,  136.0,  105.0,  137.0,  138.0,  139.0], # Car issues
  [11.0,  13.0,  12.0,  14.0,  17.0,  15.0,  16.0, 18.0,  55.0,  58.0,  45.0, 88.0], # Lapped
  [0.0], # No problem
  [77.0, 73.0, 82.0, 81.0, 62.0, 54.0, 31.0, 96.0], # Other
  [20.0] #'Spun off'
] 

def status_embed(id):
  # Create a NumPy table with a length of 6, initialized to `0.0`.
  # One-hot preparation
  _emb = np.zeros(6)
  # Upgrading the one-hot position corresponding tu the status
  for i in range(6):
    if id in stat_emb[i]:
      _emb[i] = 1
      return _emb
  # If no status was found, set the 4th position on
  _emb[4] = 1
  return _emb # Other
    # returns a binary vector of length 6 according to the category

# Convert from the one-hot to a readable string
def stat_unbed(array, retired=False):
  _a = np.copy(array)
  if (retired):
    _a[3] = 0
  _i = np.argmax(_a)
  if (_i == 0):
    return 'Accident/Collision'
  elif (_i == 1):
    return 'Car Issues'
  elif (_i == 2):
    return 'Lapped'
  elif (_i == 3):
    return 'No Problem'
  elif (_i == 4):
    return 'Other'
  elif (_i == 5):
    return 'Spun off'
  else:
    return 'something is wrong'

### Laptime embedding

In [8]:
def lapTime_embed(laptime):
    # 1) Convert the lap time into an integer count of “tenths of seconds”
    _lapTime = math.floor(laptime * 10)
    # e.g. if laptime = 123.456 s → laptime*10 = 1234.56 → floor → _lapTime = 1234

    # 2) Extract the four least-significant base-10 digits of that integer
    _lapTime_emb = []
    _ret = []
    for i in range(4):
        _lapTime_emb.append(int(_lapTime % 10))
        _lapTime = math.floor(_lapTime / 10)
    # After the loop, _lapTime_emb = [4, 3, 2, 1]  (digits, from least significant up)
    # That corresponds to the original “1234” as 4 (ones), 3 (tens), 2 (hundreds), 1 (thousands)

    # 3) Encode the hundredth of a second in 2 bits
	# - 0xx s → [0,0]
	# - 1xx s → [1,0]
	# - 2xx s → [0,1]
	# - ≥3xx s → [1,1]
    _ret = np.zeros(2)
    # We look at the most-significant of our four extracted digits:
    msd = _lapTime_emb[-1]  # here msd = 1
    if   msd == 1:
        _ret[0] = 1    # category “1xxx”
    elif msd == 2:
        _ret[1] = 1    # category “2xxx”
    elif (_lapTime_emb[-1] > 2):
        _ret[0] = 1
        _ret[0] = 1    # category “3xxx or above”  (both bits set)
    # So for our example (msd=1), _ret becomes [1, 0].

    # 4) For the remaining three digits (hundreds, tens, ones), one-hot encode each in turn
    # We want three blocks of size 10, one per digit.
    for i in range(3):          # pick idx 2 → hundreds, idx 1 → tens, idx 0 → ones
        _t = np.zeros(10)       # Init of a 10 positions one-hot
        _t[_lapTime_emb[2 - i]] = 1  # one-hot position for that digit
        _ret = np.append(_ret, _t) 
    # Appending order:
	# i=0 → digit d2 (tens)
	# i=1 → digit d1 (units)
	# i=2 → digit d0 (tenths)
    # After this, _ret has length 2 + 3*10 = 32, and looks like:
    # [<2-bit category>, <one-hot for hundreds>, <one-hot for tens>, <one-hot for ones>]
    return _ret

def lt_unbed(l_array):
  _ret = 0
  if (l_array[0] >= 0.5 and l_array[1] >= 0.5):
    _ret += 300
  elif (l_array[0] >= 0.5):
    _ret += 100
  elif (l_array[1] >= 0.5):
    _ret += 200
  _ret += np.argmax(l_array[2:12]) * 10
  _ret += np.argmax(l_array[12:22]) * 1
  _ret += np.argmax(l_array[22:32]) * 0.1
  return _ret

In [9]:
l = lapTime_embed(123.456)
print(len(l))

32


In [10]:
u = lt_unbed(l)
print(u)

123.4


### DriverId embedding

In [11]:
#drivers_short = pd.read_csv(db_dir / 'drivers_short.csv')
# Build drivers_short from all your per-race CSVs under db_dir/races/
driver_ids = set()
for year_dir in (db_dir/'races').iterdir():
    if not year_dir.is_dir(): 
        continue
    for race_csv in sorted(year_dir.glob("*.csv")):
        df_r = pd.read_csv(race_csv)
        # each race CSV has columns driverId1…driverId20
        for k in range(1, 21):
            driver_ids.update(df_r[f'driverId{k}'].unique())

drivers_short = pd.DataFrame({'driverId': sorted(driver_ids)})

# from driverId to our id
def driver_embed_idx(driverId):
    # Map an external `driverId` to an internal 1-based index for embedding lookup.
    # Returns 0 if the driverId is not found (reserved for 'unknown').

    # Query the DataFrame for rows matching the given driverId
    row = drivers_short.query(f'driverId == {driverId}').index
    # If no matching row exists, return 0 to indicate 'unknown driver'
    if row.empty:
        return 0
    # Otherwise, return the 1-based index (add 1) so that 0 remains reserved
    return row.item() + 1

def driver_unbed_idx(idx):
    # Convert an internal 1-based index back to the original external `driverId`.
    row = drivers_short.iloc[idx - 1]
    # Retrieve and return the original driverId from that row
    return row['driverId']


def driver_embed(idx):
    # Generate a one-hot encoded vector from an internal index.
    # Vector length is fixed at 130 (total drivers + 1), with position (idx-1) set to 1.

    # Initialize a zero vector of length 130
    _e = np.zeros(130)
    # Set the position corresponding to this driver to 1
    _e[idx - 1] = 1
    return _e


def driver_unbed(d_array):
    # Recover the internal index from a one-hot encoded vector.
    # np.argmax returns the first index of the max value (1 in a one-hot), 0-based
    return np.argmax(d_array) + 1  # Convert back to 1-based index

### Generating the input and output `.npy` files

In [12]:
years = range(2018, 2021)
for year in years:
    # Define the input directory for this year’s raw race CSVs
    race_dir = db_dir / 'races' / str(year)
    # If there’s no folder for that year, skip it
    if not race_dir.exists():
        print(f"Skipping year {year} (no folder: {race_dir})")
        continue

    # Define and create (if needed) the 
    # put directory for the year’s .npy files
    out_dir = db_dir / 'races_npy' / str(year)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Iterate over each race CSV file in chronological order
    for counter, csv_path in enumerate(sorted(race_dir.iterdir())):
        # Attempt to load the per-race CSV into a DataFrame
        try:
            cur_race = pd.read_csv(csv_path)
        except Exception as e:
            print(f"  ▶ Failed to read CSV '{csv_path.name}': {e}")
            continue
        
        # Number of laps is rows minus the final “no next lap” row
        nlaps = len(cur_race) - 1
        if nlaps < 1:
            print(f"  ▶ Not enough laps in '{csv_path.name}', skipping")
            continue
        
        ###
        ### Build input embeddings ###
        ###

        rows_in = []
        # For each lap (excluding the last “future” row)
        for i in range(nlaps):
            # Read the full row as floats: [circuitId, then 20 × 7 driver features]
            full = cur_race.iloc[i].values.astype(float)

            # Extract and clamp circuitId, then one-hot encode into 130 dimensions
            circuit_id = int(full[0])
            # clamp circuit_id into valid range [0,129]
            if circuit_id < 0 or circuit_id >= 130:
                circuit_id = 0

            # one-hot encode circuitId
            cur_ret = np.zeros(130)
            cur_ret[circuit_id] = 1

            # Append normalized lap in race percentage progress (0.0–1.0)
            lap_pct = i / nlaps
            cur_ret = np.append(cur_ret, lap_pct)

            # Drop circuitId and process each driver’s 7-feature block
            blocks = full[1:]  # drop circuitId
            for k in range(0, len(blocks), 7):
                # 1) driverId → embedding index → one-hot (130)
                idx = driver_embed_idx(int(blocks[k]))
                cur_ret = np.append(cur_ret, driver_embed(idx))

                # 2) driver standing → 3-bit flags (top1/2, top3, top10)
                ds = np.zeros(3)
                s = int(blocks[k+1]) - 1
                if s <= 1:  ds[0] = 1
                if s <= 3:  ds[1] = 1
                if s <= 10: ds[2] = 1
                cur_ret = np.append(cur_ret, ds)

                # 3) constructor standing → 2-bit flags (top2, top3)
                cs = np.zeros(2)
                c = int(blocks[k+2]) - 1
                if c <= 1: cs[0] = 1
                if c <= 3: cs[1] = 1
                cur_ret = np.append(cur_ret, cs)

                # 4) position → one-hot over 21 possible finishing places
                pos = np.zeros(21)
                p = min(int(blocks[k+3]) - 1, 20)
                pos[p] = 1
                cur_ret = np.append(cur_ret, pos)

                # 5) pit flag → binary
                pit_flag = 1 if blocks[k+4] == 1 else 0
                cur_ret = np.append(cur_ret, pit_flag)

                # 6) status embedding → 6-bit one-hot
                cur_ret = np.append(cur_ret, status_embed(int(blocks[k+5])))

                # 7) laptime embedding (32 dims) + 1 placeholder
                cur_ret = np.append(cur_ret, lapTime_embed(float(blocks[k+6])))
                cur_ret = np.append(cur_ret, 0)  # reserved/random

            # Collect the completed feature vector for lap i
            rows_in.append(cur_ret)

        # Stack all lap feature vectors into a 2D array and save as .npy
        in_arr = np.stack(rows_in, axis=0)
        np.save(str(out_dir / f"{counter}_input.npy"), in_arr)

        ###
        ### Build expected/output embeddings ###
        ###

        rows_out = []
        # For each lap, look at the next-lap row for expected outputs
        for i in range(nlaps):
            full_next = cur_race.iloc[i+1].values.astype(float)
            blocks = full_next[1:]
            out_vec = []

            # Decode each driver’s next-lap info
            for k in range(0, len(blocks), 7):
                # position one-hot (21 dims)
                pos = np.zeros(21)
                p = min(int(blocks[k+3]) - 1, 20)
                pos[p] = 1
                out_vec.extend(pos.tolist())

                # normalized laps-to-pit (scalar)
                out_vec.append(blocks[k+4] / nlaps)

                # status embedding (6 dims) + laptime embedding (32 dims)
                out_vec.extend(status_embed(int(blocks[k+5])).tolist())
                out_vec.extend(lapTime_embed(float(blocks[k+6])).tolist())

            # Convert this driver block into an array, then collect
            rows_out.append(np.array(out_vec))

        # Stack and save expected-output arrays
        out_arr = np.stack(rows_out, axis=0)
        np.save(str(out_dir / f"{counter}_expected.npy"), out_arr)

    # Indicate completion of this year’s processing
    print(f"Finished year {year}")

Finished year 2018
Finished year 2019
Finished year 2020


In [13]:
t = np.load(db_dir / 'races_npy/2018/0_input.npy')
t

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
len(t[0])

4051

Each data sample is a very long vector (4051) with the following features (and sizes):
- circuit id (130)
- lap number / total number of laps
- for each driver:
- - driver id (130)
- - driver standing (3)
- - constructor standing (2)
- - position (21)
- - pitting this lap OR laps/total number of laps till pitting (1) (input /output) (v1.1)
- - status (6)
- - laptime (32)
- - random (1)

In [15]:
# To make feature size consistent, we use laptimes of top 20 drivers
from pathlib import Path
import os, numpy as np, torch


class RaceDataset(torch.utils.data.Dataset):
    def __init__(self, dir_path):
        # root folder containing year subfolders of "_input.npy"/"_expected.npy"
        self.dir = Path(dir_path)

        # discover which seasons actually exist on disk
        self.available_years = sorted(
            int(p.name)
            for p in self.dir.iterdir()
            if p.is_dir() and p.name.isdigit()
        )
        if not self.available_years:
            raise FileNotFoundError(f"No year folders in {self.dir}")

        # start at the first available season
        self.year = self.available_years[0]
        self.round = 1
        self.cache = {}

        # count how many races (in-files) exist for this initial year
        year_folder = self.dir / str(self.year)
        self.cur_year = len([f for f in year_folder.iterdir() if f.name.endswith('_input.npy')])

        # load the first round’s data
        self._load_current_race()

    def _load_current_race(self):
        # Load self.cur_race from disk (and cache it).
        year_folder = self.dir / str(self.year)
        idx = min(self.round - 1, self.cur_year - 1)
        in_file  = year_folder / f"{idx}_input.npy"
        exp_file = year_folder / f"{idx}_expected.npy"

        if not in_file.exists() or not exp_file.exists():
            raise FileNotFoundError(f"Missing files: {in_file}, {exp_file}")

        # convert to str for np.load
        arr_in  = np.load(str(in_file))
        arr_exp = np.load(str(exp_file))

        self.cur_race = (arr_in, arr_exp)
        self.cache[f"{self.year}_{self.round}"] = self.cur_race

    def set_year(self, year):
        # Change to a new season (year).
        if year not in self.available_years:
            raise ValueError(f"Year {year} not found under {self.dir}")
        self.year = year
        self.round = 1
        year_folder = self.dir / str(self.year)
        self.cur_year = len([f for f in year_folder.iterdir() if f.name.endswith('_input.npy')])
        self._load_current_race()

    def set_round(self, rnd):
        # Change to a new round within the same year.
        if not (1 <= rnd <= self.cur_year):
            raise ValueError(f"Round {rnd} out of range for season {self.year}")
        self.round = rnd
        key = f"{self.year}_{self.round}"
        if key in self.cache:
            self.cur_race = self.cache[key]
        else:
            self._load_current_race()

    def next_round(self):
        # Advance to the next race, rolling over years as needed.
        # if more races in this season, just increment round
        if self.round < self.cur_year:
            self.set_round(self.round + 1)
            return

        # otherwise move to the next available season (if any)
        idx = self.available_years.index(self.year)
        if idx + 1 < len(self.available_years):
            self.year = self.available_years[idx + 1]
            self.round = 1
            year_folder = self.dir / str(self.year)
            self.cur_year = len([f for f in year_folder.iterdir() if f.name.endswith('_input.npy')])
            self._load_current_race()
        # if no next season, do nothing (stop advancing)

    def __len__(self):
        # number of laps = number of input rows
        return len(self.cur_race[0])

    def __getitem__(self, i):
        arr_in, arr_exp = self.cur_race
        return torch.from_numpy(arr_in[i]), torch.from_numpy(arr_exp[i])


In [16]:
# Dataset preparation
# only init once, cause "caching"
ds = RaceDataset(db_dir / 'races_npy/')

In [17]:
# Activation of the cuDNN optimisations
cudnn.benchmark = True

In [18]:
ds[0]

(tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64),
 tensor([1., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64))

In [19]:
len(ds[0][0])

4051

In [20]:
len(ds[0][1])

1200

In [21]:
ds[0][1][29*11+28]

tensor(0., dtype=torch.float64)

### Helper function for displaying dataset input/output

In [22]:
def pos_df(lap_in, out, num_of_laps=1):
    # prepare list to collect each driver's row
    rows = []
    _lap = lap_in.detach().cpu().numpy()
    _o   = out.detach().cpu().numpy()

    # decode circuit info
    _name, _loc, _country = circuit_info(np.argmax(_lap[:130]))

    for i in range(20):
        # decode driver index from one-hot
        start = 131 + i * 196
        driver_block = _lap[start : start + 130]
        _d_idx = driver_unbed_idx(driver_unbed(driver_block))

        # lookup driver metadata
        _num, _code, _fn, _ln, _, _ = driver_info(_d_idx)

        # position
        pos_block = _o[i*60 : i*60 + 21]
        _pos = int(np.argmax(pos_block)) + 1

        # laps-till-pitting
        _pitting = float(_o[i*60+21]) * num_of_laps
        if _pitting == 0:
            _pitting = 'NA'

        # status
        retired = (_pos == 21)
        status_block = _o[i*60 + 22 : i*60 + 28]
        _status = stat_unbed(status_block, retired)

        # lap time
        laptime_block = _o[i*60 + 28 : i*60 + 60]
        _time = lt_unbed(laptime_block)

        # append this row to our list
        rows.append({
            'code': _code,
            'driver': f'{_fn} {_ln}',
            'position': _pos,
            'laps till pitting': _pitting,
            'status': _status,
            'laptime': _time
        })

    # build DataFrame in one go and sort
    df = pd.DataFrame(rows, columns=[
        'code', 'driver', 'position', 'laps till pitting', 'status', 'laptime'
    ])
    df = df.sort_values(by=['position', 'laptime'], ascending=[True, False])
    return _name, _loc, _country, df


In [23]:
# Test with the 1st turn of the 1st race of 2018
ds.set_year(2018)
ds.set_round(1)
_, _, _, df = pos_df(ds[1][0], ds[1][1])

In [24]:
# Display the test result
df

,code,driver,position,laps till pitting,status,laptime
0,HAM,Lewis Hamilton,1,NA,No Problem,83.1
1,BOT,Valtteri Bottas,2,NA,No Problem,83.7
3,VET,Sebastian Vettel,3,NA,No Problem,84.2
2,RAI,Kimi Räikkönen,4,NA,No Problem,84.0
6,VER,Max Verstappen,5,NA,No Problem,84.6
5,GAS,Pierre Gasly,6,NA,No Problem,85.6
8,MAG,Kevin Magnussen,7,NA,No Problem,86.1
4,SAI,Carlos Sainz,8,NA,No Problem,86.4
7,HAR,Brendon Hartley,9,NA,No Problem,86.5
12,HUL,Nico Hülkenberg,10,NA,No Problem,86.7


In [25]:
# Check the size of the input vector
ds[0][0].shape

torch.Size([4051])

In [26]:
try:
    # if I did `round = ...`, this will delete it
    del round
except NameError:
    # no user-defined `round` to delete
    pass

In [27]:
# Returns a tensor with the size of in but content of out
def out_to_in(in_, out_, random=False, num_of_laps=50):
    _ret = in_.detach().clone().numpy().reshape(-1)
    _o   = out_.detach().clone().numpy().reshape(-1)
    j = 0

    for i in range(0, 20):
      _ret[131 + i*196 + 135: i*196 + 131 + 195] = _o[j*60: (j+1) * 60]
      # pred_norm_pit = _o[j*60 + 21] # the 22nd float in the 60`
      if (round(_o[j*60 + 21] * num_of_laps) <= 1):
        _ret[131 + i*196 + 130 + 26] = 1
      else:
        _ret[131 + i*196 + 130 + 26] = 0
      if (random):
        _ret[131 + i*196 + 195] = np.random.uniform(0, 20)
        
    j += 1

    return torch.from_numpy(_ret).float()

In [28]:
# Perform the out to in switch
outfromin = out_to_in(ds[0][0], ds[0][1])

In [29]:
# Check the new tensor look
print(outfromin)

tensor([0., 0., 0.,  ..., 0., 0., 0.])


In [30]:
# Generate the next lap status
_, _, _, df2 = pos_df(outfromin, ds[0][1])


In [31]:
# Display the test result
df2

,code,driver,position,laps till pitting,status,laptime
0,HAM,Lewis Hamilton,1,NA,No Problem,86.7
1,BOT,Valtteri Bottas,2,NA,No Problem,87.7
3,VET,Sebastian Vettel,3,NA,No Problem,88.7
2,RAI,Kimi Räikkönen,4,NA,No Problem,89.7
6,VER,Max Verstappen,5,NA,No Problem,90.8
5,GAS,Pierre Gasly,6,NA,No Problem,91.7
8,MAG,Kevin Magnussen,7,NA,No Problem,92.4
4,SAI,Carlos Sainz,8,NA,No Problem,93.3
7,HAR,Brendon Hartley,9,NA,No Problem,94.1
12,HUL,Nico Hülkenberg,10,NA,No Problem,94.6


## Model

In [32]:
class RacePredictionModel(nn.Module):
    def __init__(self, input_size, output_size, lstm_hiddens, lstm_layers, dropout):
        # In our test : 4051         1200          1200           2         0.2
        # Call the parent class constructor
        # - this is required to properly initialize the module
        # - it also allows us to use the `self` keyword to access module methods/attributes
        # - this is a standard pattern for all PyTorch modules
        super(RacePredictionModel, self).__init__()

        # Store configuration parameters
        self.input_size = input_size        # dimensionality of each input vector
        self.lstm_layers = lstm_layers      # number of stacked LSTM layers
        self.lstm_hiddens = lstm_hiddens          # hidden state size of each LSTM layer

        # Define the LSTM module
        # - input_size: number of features in the input at each time step = Number of positions in the input line
        # - output_size: number of features in the output at each time step = Number of positions in the output line
        # - lstm_hiddens: size of the hidden state in the LSTM - each layer has 1 200 hidden units, so all of your hidden‐ and cell‐state tensors are shaped
        # - lstm_layers: number of stacked LSTM layers - two stacked LSTMs is a common default for sequence-to-sequence or time-series tasks
        # - dropout: probability of randomly (20% here) zeroing outputs to force the model to not rely too heavily on any single hidden‐unit or pattern in layer-1
        # - batch_first=True: input/output tensors shaped as (batch, seq, feature)
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=lstm_hiddens,
            num_layers=lstm_layers,
            dropout=dropout,
            batch_first=True
        )

        # Define a fully-connected layer to map from LSTM hidden state to output vector
        # - takes the LSTM hidden dimension and produces the desired output_size
        self.fc = nn.Linear(lstm_hiddens, output_size)

        # Initialize the FC layer weights with Xavier (Glorot) uniform initialization
        #   This initialization scheme keeps the scale of activations and gradients roughly 
        #   the same across layers when you start training, which helps with convergence and 
        #   avoids vanishing/exploding signals in deep networks.
        nn.init.xavier_uniform_(self.fc.weight.data)

        # Initialize LSTM parameters:
        # - weight matrices with Xavier uniform
        # - bias terms to zero
        for name, params in self.lstm.named_parameters():
            if name[:6] == 'weight':
                nn.init.xavier_uniform_(params)
            elif name[:4] == 'bias':
                nn.init.constant_(params, 0.0)

    def zero_states(self, batchsize=1):
        # Create zero-initialized hidden and cell states for the LSTM.
        # - batchsize: the number of parallel sequences to process in batch.
        # Returns a tuple (hidden_state, cell_state) each of shape
        # (num_layers, batchsize, lstm_hiddens).
        hidden_state = torch.zeros(self.lstm_layers, batchsize, self.lstm_hiddens)
        cell_state = torch.zeros(self.lstm_layers, batchsize, self.lstm_hiddens)
        return (hidden_state, cell_state)

    def forward(self, ins, prev_states=None):
        # Forward pass through the model.
        # - ins: input tensor of shape (batchsize, seq_len, input_size)
        # - prev_states: optional tuple of (hidden_state, cell_state) from a previous pass
        # Returns:
        #   - outs: output tensor of shape (batchsize, seq_len, output_size)
        #   - next_states: tuple (hidden_state, cell_state) after processing `ins`

        # Pass the input sequence through the LSTM
        lstm_outs, next_states = self.lstm(ins, prev_states)
        # Apply the fully-connected layer to each time step's output
        outs = self.fc(lstm_outs)
        return outs, next_states

### Helper functions for training #1

In [33]:
# cuDNN optimisations activation
cudnn.benchmark = True

In [34]:
# Device selection
# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Snapshot folder
sd_dir = Path(r"E:\Dropbox\Informatique\Holberton\F1_Project\sd")
sd_dir.mkdir(parents=True, exist_ok=True)

Using device: cuda


In [ ]:

# Trains model on data from 2001 to 2024, 2024 is reserved for testing
# Method 1
# Training procedure:
#   for each race:
#     set zero states,
#     for each lap:
#       feed input from dataset,
#       calculate loss from output,
#       back propagate,

# Trains model on data from 2001 to 2019, 2024 is reserved for testing
# Method 1: truncated back-propagation through time with state detachment

# Model initialization:
#   - Input features: 4051
#   - Two LSTM layers with 1200 units each
#   - Output features: 2
#   - Dropout: 0.2
model = RacePredictionModel(4051, 1200, 1200, 2, 0.2).to(device)

# Mean Squared Error (MSE) loss module is a standard regression loss function
crit  = nn.MSELoss().to(device)

# Adam : default optimizer with learning rate 0.001
# Learning rate scheduler: multiply LR by 0.2 every epoch ----------> Aggressive decay, makes sense for LSTM
opt   = optim.Adam(model.parameters(), lr=0.001)

#   - step_size: number of epochs before decay
#       - step_size=1: decay every epoch    
#   - gamma: multiplicative factor for decay
#       - gamma=0.2: decay factor (0.2 means 80% reduction)
sched = optim.lr_scheduler.StepLR(opt, step_size=1, gamma=0.2)

###
### Training and updating functions ###
###

def run_train(model, ds, crit, opt, sched, device):
    # Trains the model on available seasonal data (up to the year before 2024),
    # and displays a progress bar per race.
    model.train()
    total_loss, count = 0.0, 0

    # Build list of (year, round) tuples for all races prior to 2024
    race_list = []
    for y in ds.available_years:
        # Stop when we reach 2024 (reserved for testing)
        if y >= 2024:
            break
        ds.set_year(y)
        # ds.cur_year a bien été mis à jour par set_year
        for rnd in range(1, ds.cur_year + 1):
            race_list.append((y, rnd))

    # Iterate over each race with a progress bar
    for (y, rnd) in tqdm(race_list, desc="Training epoch", unit="race"):
        ds.set_year(y)
        ds.set_round(rnd)

        # DataLoader for this race, batch_size=1 (one lap at a time), no multiprocessing on Windows
        race_loader = DataLoader(
            ds,
            batch_size=1,
            shuffle=False,
            num_workers=0,
            pin_memory=(device.type == 'cuda')
        )

        # Initialize hidden and cell states for the LSTM
        h0, c0 = model.zero_states()
        st = (h0.to(device), c0.to(device))

        # Loop over each lap in the race
        for inp_batch, exp_batch in race_loader:
            # Prepare input and target tensors: add sequence dimension and cast to float
            x = inp_batch.unsqueeze(1).float().to(device, non_blocking=True)
            y = exp_batch.unsqueeze(1).float().to(device, non_blocking=True)

            # Zero gradients from previous iteration
            opt.zero_grad()
            # Forward pass through model, updating hidden states
            out, st = model(x, st)
            # Compute loss
            loss = crit(out, y)
            # Backpropagate gradients
            loss.backward()
            # Update model parameters
            opt.step()

            # Detach hidden states to truncate backprop through time (BPTT)
            st = (st[0].detach(), st[1].detach())

            # Accumulate loss for reporting
            total_loss += loss.item()
            count += 1

    # Step the LR scheduler after processing all races in this epoch
    sched.step()

    # Return average loss per lap
    return total_loss / count if count > 0 else float('nan')


def run_test(model, ds, crit, device):
    # Set model to evaluation mode (disables dropout, etc.)
    model.eval()
    total, count = 0, 0

    # Choose test year: first available year >= 2024, otherwise fallback to latest year
    test_years = [y for y in ds.available_years if y >= 2024]
    if test_years:
        year0 = test_years[0]
    else:
        year0 = ds.available_years[-1]   # fallback
    ds.set_year(year0)
    ds.set_round(1) # tests start from first round

    # DataLoader for testing single race sequentially (no batching across races)
    race_loader = DataLoader(
        ds,
        batch_size=1,
        shuffle=False,
        num_workers=0,
        pin_memory=(device.type == 'cuda')
    )
     # Initialize hidden states
    hs, cs = model.zero_states()
    st = (hs.to(device), cs.to(device))

    # Disable gradient computation for efficiency
    with torch.no_grad():
        for inp_batch, exp_batch in race_loader:
            # Prepare tensors
            x = inp_batch.unsqueeze(1).float().to(device, non_blocking=True)
            y = exp_batch.unsqueeze(1).float().to(device, non_blocking=True)
            # Forward pass
            out, st = model(x, st)
            # Accumulate test loss
            total += crit(out, y).item()
            count += 1

    # Return average test loss per lap
    return total / count if count > 0 else float('nan')



def run_all(model, ds, crit, opt, sched, device, epochs=1):    # epochs=10
    # Loop over number of epochs specified
    for epoch in range(epochs):
        print(f"Epoch {epoch}")
        # Training phase
        tr_loss = run_train(model, ds, crit, opt, sched, device)
        # Testing phase
        te_loss = run_test(model, ds, crit, device)
        # Report metrics
        print(f"Epoch {epoch} — Train Loss: {tr_loss:.4f} | Test Loss: {te_loss:.2f}")

        # Save checkpoint every 5 epochs
        if epoch % 5 == 0:
            ckpt = sd_dir / f"checkpoint-epoch{epoch}-loss{te_loss:.2f}.pth"
            torch.save(model.state_dict(), ckpt)

     # Save final model after last epoch
    final_path = sd_dir / f"final-epoch{epochs-1}-loss{te_loss:.2f}.pth"
    torch.save(model.state_dict(), final_path)



### Training

In [36]:
# pick GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

epochs = 1
run_all(model, ds, crit, opt, sched, device, epochs)

Using device: cuda
Epoch 0


Training epoch:   0%|          | 0/42 [00:00<?, ?race/s]

Epoch 0 — Train Loss: 0.0546 | Test Loss: 0.10


In [37]:
run_test(model, ds, crit, device)


0.10251132524769072

In [ ]:
#cpu = torch.device('cpu')

# Example of how the model was previously loaded from a specific checkpoint (commented out)
# model.load_state_dict(torch.load('/content/drive/My Drive/f1ml/sd/47.1-5-loss-0.07.pth'))

# Gather all files ending with .pth in the checkpoint directory
ckpts = list(sd_dir.glob("*.pth"))
if not ckpts:
    # If no checkpoints are found, raise an error with a descriptive message
    raise FileNotFoundError(f"Aucun checkpoint trouvé dans {sd_dir}")

# Sort the checkpoint files by their last modification time (oldest first)
ckpts = sorted(ckpts, key=lambda p: p.stat().st_mtime)
# Select the most recently modified checkpoint
latest = ckpts[-1]
print(f"🔄 Chargement du checkpoint : {latest.name}")

# Load the model weights from the latest checkpoint.

# If we're on CPU, ensure tensors are mapped to the correct device
#model.load_state_dict(torch.load(latest, map_location=device))

# Switch the model to evaluation mode to disable dropout, batchnorm updates, etc.
model.eval()

# Set dataset to use the 2024 season and the 10th round
ds.set_year(2024)
ds.set_round(10)

# Retrieve the input features and expected output for the 10th lap
p, n = ds[10]
# Move input tensor to the computation device (CPU or GPU)
p = p.to(device)

# Initialize hidden and cell states for the model's LSTM layers
hid_state, cell_state = model.zero_states()
states = (hid_state.to(device), cell_state.to(device))

# Perform a forward pass through the model:
# - Unsqueeze twice to add batch and sequence dimensions
# - Convert input to float
out, s = model(p.unsqueeze(0).unsqueeze(0).float(), states)

# Remove the added batch and sequence dimensions from the output
out = out.squeeze().squeeze()
print(out)

# Optionally, convert predictions to a pandas DataFrame using CPU tensors
# pos_df(p.to(cpu), out.to(cpu))
name, loc, country, df = pos_df(p.cpu(), out.cpu())
print(df)

🔄 Chargement du checkpoint : final-epoch0-loss0.10.pth
tensor([0.9014, 0.0232, 0.1339,  ..., 0.0722, 0.1817, 0.0025], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
   code              driver  position  laps till pitting      status  laptime
0   HAM      Lewis Hamilton         1           0.024014  No Problem    101.5
1   VER      Max Verstappen         2          -0.023678  No Problem    101.5
2   BOT     Valtteri Bottas         3          -0.000930  No Problem    100.4
19  LAT     Nicholas Latifi         4           0.002529  No Problem    101.8
3   PER        Sergio Pérez         4          -0.001795  No Problem    101.2
4   RIC    Daniel Ricciardo         5          -0.013361  No Problem    102.3
5   SAI        Carlos Sainz         7           0.003256  No Problem    103.0
9   LEC     Charles Leclerc         8           0.022530  No Problem    102.1
12  RUS      George Russell         9          -0.024174  No Problem    103.0
8   GAS        Pierre Gasly        10          -0.

In [39]:
# 1) List all the checkpoints .pth
ckpts = list(sd_dir.glob("*.pth"))
if not ckpts:
    raise FileNotFoundError(f"Aucun fichier .pth trouvé dans {sd_dir}")
# 2) Choose the most recent one ordered by update date
latest = max(ckpts, key=lambda p: p.stat().st_mtime)
print(f"🔄 Chargement du checkpoint : {latest.name}")
# 3) Load on CPU (or use `map_location=device` for GPU load)
model.load_state_dict(torch.load(latest, map_location="cpu"))
model.to("cpu")

🔄 Chargement du checkpoint : final-epoch0-loss0.10.pth


RacePredictionModel(
  (lstm): LSTM(4051, 1200, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=1200, out_features=1200, bias=True)
)

In [40]:
#torch.save(model.state_dict(), (sd_dir / f'{versionId}-{epochs}-loss-{te_loss:.2f}.pth'))

In [ ]:
# pick GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

# 1) List all the checkpoints .pth
ckpts = list(sd_dir.glob("*.pth"))
if not ckpts:
    raise FileNotFoundError(f"Aucun checkpoint trouvé dans {sd_dir}")
# 2) Choose the most recent one ordered by update date
latest = max(ckpts, key=lambda p: p.stat().st_mtime)
print(f"🔄 Chargement du checkpoint : {latest.name}")
# 3) Load on CPU (or use `map_location=device` for GPU load)
model.load_state_dict(torch.load(latest, map_location=device))

model.eval()
# On se positionne sur la course n°30 de la saison 2024
ds.set_year(2024)
ds.set_round(12)

# On récupère le premier tour "p" et la vérité n
p, n = ds[0]
p = p.to(device)

hid_state, cell_state = model.zero_states()
states = (hid_state.to(device), cell_state.to(device))
out, s = model(p.unsqueeze(0).unsqueeze(0).float(), states)
out = out.squeeze().squeeze()
print(out)
#pos_df(p.to(cpu), out.to(cpu))
name, loc, country, df = pos_df(p.cpu(), out.cpu())
print(df)

# --- Démo d'un petit batch de 2 exemples (batch_inputs) ---

batch_inputs = torch.cat([
    ds[0][0].unsqueeze(0).unsqueeze(0),
    ds[1][0].unsqueeze(0).unsqueeze(0)
], dim=0)  # shape = [2, 1, 4051]

# 1) Réinit hidden & cell POUR batch_inputs (batchsize=2)
batch_size = batch_inputs.size(0)    # -> 2
h0_b, c0_b = model.zero_states(batchsize=batch_size)
states_b = (h0_b.to(device), c0_b.to(device))

# 2) Passage avant sur batch_inputs
o_batch, s_batch = model(batch_inputs.float().to(device), states_b)
print("→ batch_inputs → sortie shape:", o_batch.shape)
print("→ batch_inputs → hidden shape:", s_batch[0].shape)  # (num_layers, 2, hidden)

# --- Maintenant on revient à un seul exemple pour p ---

# 3) Réinit hidden & cell POUR p (batchsize=1)
h0_p, c0_p = model.zero_states(batchsize=1)
states_p = (h0_p.to(device), c0_p.to(device))

# 4) Premier passage sur p
out, states_p = model(p.unsqueeze(0).unsqueeze(0).float(), states_p)
out = out.squeeze(0).squeeze(0)

# 5) Boucle de prédiction sur 50 tours
n_tours = 50
num_of_laps = len(ds)
for i in range(n_tours):
    # Génération de l'entrée suivante à partir de la sortie
    inp = out_to_in(
        p.cpu(),
        out.cpu(),
        random=True,
        num_of_laps=len(ds)
    ).to(device)

    # Passage avant
    out, states = model(inp.unsqueeze(0).unsqueeze(0).float(), states)
    out = out.squeeze(0).squeeze(0)

    # Décodage en DataFrame
    lap_in = inp  # plutôt que de réutiliser p
    predicted_out = out

    name, loc, country, df = pos_df(lap_in.cpu(), predicted_out.cpu(), num_of_laps)
    # n’affiche que si on est au dernier tour
    if i == n_tours - 1:
        print(f"Tour prédit {i+1} — {name} ({loc}, {country})")
        display(df)

# Pour garder le dernier DataFrame
df_final = df.copy()
#display(df_final)


Using device: cuda
🔄 Chargement du checkpoint : final-epoch0-loss0.10.pth
tensor([ 0.8216,  0.0575,  0.1221,  ...,  0.1209,  0.2131, -0.0087],
       device='cuda:0', grad_fn=<SqueezeBackward0>)
   code              driver  position  laps till pitting      status  laptime
0   HAM      Lewis Hamilton         1           0.022969  No Problem    101.5
1   BOT     Valtteri Bottas         2          -0.035418  No Problem    101.5
2   VER      Max Verstappen         3          -0.020580  No Problem    100.7
19  LAT     Nicholas Latifi         4           0.008332  No Problem    101.8
4   PER        Sergio Pérez         5          -0.024003  No Problem    102.3
3   LEC     Charles Leclerc         6           0.017454  No Problem    101.2
5   ALB     Alexander Albon         7           0.026296  No Problem    103.5
9   RIC    Daniel Ricciardo         8           0.049621  No Problem    102.1
12  KVY        Daniil Kvyat         9          -0.004953  No Problem    103.0
8   GAS        Pierre Gas

,code,driver,position,laps till pitting,status,laptime
0,HAM,Lewis Hamilton,1,1.534464,No Problem,101.5
1,BOT,Valtteri Bottas,2,-0.269804,No Problem,101.7
2,VER,Max Verstappen,3,-0.639818,No Problem,100.8
19,LAT,Nicholas Latifi,4,-2.655934,No Problem,101.0
4,PER,Sergio Pérez,5,-1.086303,No Problem,102.4
3,LEC,Charles Leclerc,6,-1.156822,No Problem,101.0
5,ALB,Alexander Albon,7,-3.414034,No Problem,103.1
9,RIC,Daniel Ricciardo,8,0.152531,No Problem,103.1
12,KVY,Daniil Kvyat,9,1.847972,No Problem,103.0
8,GAS,Pierre Gasly,10,-1.291542,No Problem,103.4


In [42]:
#pos_df(p.to(cpu), n.to(cpu))

In [43]:
c = torch.cat((ds[0][0].unsqueeze(0).unsqueeze(0), ds[1][0].unsqueeze(0).unsqueeze(0)), 0)

In [44]:
c.shape

torch.Size([2, 1, 4051])

In [45]:
#o, s = model(c.to(device).float(), states)
# 1) Calcul du batch size
batch_size = c.size(0)  # par ex. 2 si c.shape[0] == 2

# 2) Réinitialisation des hidden et cell states
h0, c0 = model.zero_states(batchsize=batch_size)
states = (h0.to(device), c0.to(device))

# 3) Appel du modèle avec les bons états
o, s = model(c.to(device).float(), states)

In [46]:
o.shape

torch.Size([2, 1, 1200])

In [47]:
s[0].shape

torch.Size([2, 2, 1200])

In [48]:
s[1].shape

torch.Size([2, 2, 1200])

In [49]:
# Retrieve initial hidden and cell states for a batch of size 1 from the model
h0_s, c0_s = model.zero_states(batchsize=1)

# Move the initial LSTM states to the appropriate device (CPU or GPU)
states = (h0_s.to(device), c0_s.to(device))

# Select the first sample from the dataset:
#  - ds[0] gives a (features, target) tuple
#  - ds[0][0] extracts the feature tensor for the first lap
#  - unsqueeze(0) twice to add both batch and time‐step dimensions
#  - cast to float and move to device for computation
oo, ss = model(
    ds[0][0]
      .unsqueeze(0)    # add batch dimension -> shape becomes (1, features)
      .unsqueeze(0)    # add time dimension -> shape becomes (1, 1, features)
      .float()         # ensure tensor is float32
      .to(device),     # send tensor to CPU or GPU
    states             # initial hidden and cell states
)

# Print the shapes of:
#  - oo: the model’s output tensor for this single time step
#  - ss[0]: the hidden state tensor after this forward pass
print("oo:", oo.shape, "ss_hidden:", ss[0].shape)

oo: torch.Size([1, 1, 1200]) ss_hidden: torch.Size([2, 1, 1200])


In [50]:
oo.shape

torch.Size([1, 1, 1200])

In [51]:
ss[0].shape

torch.Size([2, 1, 1200])

In [52]:
len(ss)

2